## Data Loading

In [1]:
import gzip, json
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

data = []

for review in parse("Software.json.gz"):
  data.append(review)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

indices = np.arange(len(data))
indices_train, indices_test = train_test_split(indices, test_size=0.1, random_state=42)
indices_train, indices_val = train_test_split(indices_train, test_size=0.1/0.9, random_state=42)

In [3]:
data_np = np.array(data)
data_train = data_np[indices_train]
data_val = data_np[indices_val]
data_test = data_np[indices_test]

In [4]:
y_train = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_train]
y_val = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_val]
y_test = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_test]

## Naive Bayes

## Non-text (review) features

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# fit the ID encoder on full data set
userID_str = np.array([[d['reviewerID']] for d in data])
userID_encoder = LabelEncoder()
userID_encoder.fit(userID_str.squeeze())
userID_one_hot_encoder = OneHotEncoder()
userID_one_hot_encoder.fit(userID_str)

itemID_str = np.array([[d['asin']] for d in data])
itemID_encoder = LabelEncoder()
itemID_encoder.fit(itemID_str.squeeze())
itemID_one_hot_encoder = OneHotEncoder()
itemID_one_hot_encoder.fit(itemID_str)

OneHotEncoder()

In [6]:
import textstat
import os
import pickle 

if os.path.isfile('readability_scores.pickle'):
    with open('readability_scores.pickle', 'rb') as handle:
        readability_scores = pickle.load(handle)
    
else:
    # Might take 2-3 mins, please be patient!
    readability_scores = {}
    for d in data:
        if 'reviewText' in d:
            readability_scores[d['reviewText']] = textstat.flesch_reading_ease(d['reviewText'])

In [7]:
def get_meta_features(d, length=False, rating=False, readability=False, verified=False, userID=False, itemID=False):
    feature_vec = []
    
    if length:
        if 'reviewText' not in d:
            feature_vec.append(0)
        else:
            feature_vec.append(len(d['reviewText']))
            
    if rating:
        feature_vec.append(d['overall'])
    
    if readability:
        if 'reviewText' not in d:
            feature_vec.append(100)
        else:
            feature_vec.append(readability_scores[d['reviewText']])
            
    if verified:
        if 'verified' not in d:
            feature_vec.append(0)
        elif d['verified']:
            feature_vec.append(1)
        else:
            feature_vec.append(0)
            
    if userID:
        feature_vec.extend(list(userID_one_hot_encoder.transform([[d["reviewerID"]]]).toarray()[0]))
                                          
    if itemID:
        feature_vec.extend(list(itemID_one_hot_encoder.transform([[d["asin"]]]).toarray()[0]))      
        
    return feature_vec
        

In [8]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def test_metrics(clf, X_t, y_t):
    y_pred = clf.predict(X_t)
    precision, recall, _, _ = precision_recall_fscore_support(y_t, y_pred, average='binary')
    accuracy = accuracy_score(y_t, y_pred)
    return accuracy, precision, recall, 2*precision*recall/(precision+recall)

In [9]:
X_train = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_train]
X_val = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_val]
X_test = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_test]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(X_train)

In [10]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            # print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                # print('INFO: Early stopping')
                self.early_stop = True

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import cross_entropy
from tqdm import tqdm
from torch.autograd import Variable
from math import ceil
from torch.optim.lr_scheduler import StepLR


class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5, l2_reg=0.01, tol=1e-3, lr=1e-1):
        super(MLPClassifier, self).__init__()

        # Check if hidden_size is a tuple
        if isinstance(hidden_size, tuple):
            # Create a list of linear layers for each hidden size
            layers = []
            prev_size = input_size
            for size in hidden_size:
                layers.append(nn.Linear(prev_size, size))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout_prob))
                prev_size = size
            self.hidden_layers = nn.Sequential(*layers)
        else:
            # Single hidden layer
            self.hidden_layers = nn.Sequential(
                nn.Linear(input_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(p=dropout_prob)
            )

        self.fc_out = nn.Linear(prev_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        self.l2_reg = l2_reg
        self.tol = tol
        self.class_weights = torch.tensor([0, 0], requires_grad=False)
        self.lr = lr

    def forward(self, x):
        # Input layer

        # Hidden layers
        for layer in self.hidden_layers:
            x = layer(x)

        # Output layer
        x = self.fc_out(x)  # No activation function for binary classification

        return self.softmax(x)
    
    def calculate_loss(self,x,y):
        device = torch.device('cuda')
        l2_reg_term = torch.nn.utils.parameters_to_vector(self.parameters()).norm(p=2)
        one_hot_encoded = torch.zeros(len(y), 2).to(device)
        one_hot_encoded.scatter_(1, y.unsqueeze(1), 1)
        y_pred = self.forward(x.float())
        return cross_entropy(y_pred, one_hot_encoded, weight=self.class_weights.to('cuda')) + self.l2_reg * l2_reg_term


    def fit(self, X_train, y_train, x_val, y_val, epochs=100, use_gpu=True):
        # Check if GPU is available and decide whether to use it
        use_gpu = use_gpu and torch.cuda.is_available()
        device = torch.device('cuda' if use_gpu else 'cpu')
        self.to(device)

        optimizer = optim.Adam(self.my_trainable_params(), lr=self.lr)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=3, factor=0.5, min_lr=1e-6)
        es = EarlyStopping(patience=10)



        print(self.class_weights)

        # x_train_tensor = torch.tensor(X_train).to(device)
        # y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

        x_train_tensor = torch.tensor(X_train) if not isinstance(X_train, torch.Tensor) else X_train
        x_train_tensor = x_train_tensor.to(device)
        
        y_train_tensor = torch.tensor(y_train, dtype=torch.long) if not isinstance(X_train, torch.Tensor) else y_train
        y_train_tensor = y_train_tensor.to(device)

        # Calculate class weights if not provided
        class_counts = torch.bincount(y_train_tensor)
        self.class_weights = 1.0 - (class_counts.float() / torch.sum(class_counts))
        self.class_weights.to(device)


        x_valid = torch.tensor(x_val).to(device)
        y_valid = torch.tensor(y_val).to(device)

        prev_loss = 1e12
        loss = 1e6

        epoch = 0
        while epoch < epochs:
            loss = self.calculate_loss(x_train_tensor, y_train_tensor)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            val_loss = self.calculate_loss(x_valid, y_valid)
            lr_scheduler.step(val_loss)
            es(val_loss)

            if es.early_stop:
                break

            epoch += 1

        print(f"Training stopped after {epoch} epochs.")

    def my_trainable_params(self):
        trainable_params = []
        for layer in self.hidden_layers:
            # print(type(layer))
            if isinstance(layer, nn.Linear):
                trainable_params.extend(layer.parameters())

        if isinstance(self.fc_out, nn.Linear):
            trainable_params.extend(self.fc_out.parameters())
        return trainable_params

    def predict(self, x, use_gpu=True):
        # Move inputs to GPU if available
        device = torch.device('cuda' if use_gpu else 'cpu')
        inputs = Variable(torch.FloatTensor(x)).to(device)
        prediction = self.forward(inputs)
        return torch.argmax(prediction.data.cpu(), 1).numpy() if use_gpu else torch.argmax(prediction.data, 1).numpy()

# Example usage:
# classifier = MLPClassifier(input_size=..., hidden_size=..., output_size=...)
# classifier.fit(X_train, y_train, epochs=..., batch_size=..., use_gpu=True)
# predictions = classifier.predict(X_test, use_gpu=True)


In [12]:
# Logistic regression
# from sklearn.linear_model import LogisticRegression

clf = MLPClassifier(input_size=len(X_train[0]),hidden_size=(100,10,),output_size=2)
clf.fit(scaler.transform(X_train), y_train, scaler.transform(X_val), y_val, epochs=100)
test_metrics(clf, scaler.transform(X_test), y_test)

tensor([0, 0])
Training stopped after 65 epochs.


(0.7287785129723141,
 0.5081853570095451,
 0.7463972431077694,
 0.6046762475809777)

In [13]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([265249, 102299]))

In [14]:
# Define the hyperparameter grid
param_grid = {
    'hidden_size': [(5,), (64,), (128,), (64, 32), (64, 32, 16)],
    'l2_reg': [0.001, 0.01, 0.1, 1, 10, 100],
    # 'tol': [1e-3, 1e-4, 1e-5]
}

In [15]:
from sklearn.model_selection import ParameterGrid



# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_notext = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(scaler.transform(X_train), y_train, scaler.transform(X_val), y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, scaler.transform(X_test), y_test)
    print(metrics)
    scores_notext.append({'params':params, 'scores':metrics})
    bar.set_postfix()


  0%|          | 0/30 [00:00<?, ?it/s]

Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])


  3%|▎         | 1/30 [00:01<00:29,  1.03s/it]

Training stopped after 47 epochs.
(0.7353082012885251, 0.5183727828561051, 0.6706610275689223, 0.5847645713115034)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 18 epochs.


  7%|▋         | 2/30 [00:01<00:26,  1.07it/s]

...

Training stopped after 75 epochs.
(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_591361/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
 97%|█████████▋| 29/30 [00:48<00:01,  1.97s/it]

Training stopped after 100 epochs.
(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])


100%|██████████| 30/30 [00:50<00:00,  1.69s/it]

Training stopped after 100 epochs.
(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [16]:
# with open('mlp_notext.json', 'w') as f:
#     json.dump(scores_notext, f)

## Text Features

In [17]:
train_documents = []
for d in data_train: 
    strs = []
    if 'summary' in d:
        strs.append(d['summary'])
    if 'reviewText' in d:
        strs.append(d['reviewText'])
    train_documents.append(" ".join(strs))

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(stop_words='english', max_features=300)
bow_vectorizer.fit(train_documents)

CountVectorizer(max_features=300, stop_words='english')

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=300)
tfidf_vectorizer.fit(train_documents)

TfidfVectorizer(max_features=300, stop_words='english')

In [20]:
w2v_features = np.load('text-feature_word2vec.npz')['word2vec']
w2v_dict = {}

count = 0 
for d in data:
    if 'reviewText' in d:
        w2v_dict[d['reviewText']] = w2v_features[count, :]
    else:
        w2v_dict[''] =  w2v_features[count, :]
    count+=1

In [21]:
all_docs = []
for d in data: 
    strs = []
    if 'summary' in d:
        strs.append(d['summary'])
    if 'reviewText' in d:
        strs.append(d['reviewText'])
    all_docs.append(" ".join(strs))
    

In [22]:
tfidf_vecs = tfidf_vectorizer.transform(all_docs).toarray()

tfidf_dict = {}
count = 0
for d in data:
    if 'reviewText' in d:
        tfidf_dict[d['reviewText']] = tfidf_vecs[count, :]
    else:
        tfidf_dict[''] =  tfidf_vecs[count, :]
    count+=1

In [23]:
bow_vecs = bow_vectorizer.transform(all_docs).toarray()

bow_dict = {}
count = 0
for d in data:
    if 'reviewText' in d:
        bow_dict[d['reviewText']] = bow_vecs[count, :]
    else:
        bow_dict[''] =  bow_vecs[count, :]
    count+=1

In [24]:
def get_text_features(d, BoW=False, tfidf=False, w2v=False):
    
    feature_vec = []
    
    if 'reviewText' in d:
        reviewText = d['reviewText']
    else:
        reviewText = ''
    
    if BoW:
        feature_vec.extend(bow_dict[reviewText].tolist())
    if tfidf:
        feature_vec.extend(tfidf_dict[reviewText].ravel())
    if w2v:
        feature_vec.extend(w2v_dict[reviewText].tolist())
        
    return feature_vec
       

In [25]:
X_train = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_train]
X_val = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_val]
X_test = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_test]

In [26]:
from sklearn.model_selection import ParameterGrid

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_bow = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_bow.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_bow.json', 'w') as f:
    json.dump(scores_bow, f)

  0%|          | 0/30 [00:00<?, ?it/s]

Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 77 epochs.


  3%|▎         | 1/30 [00:13<06:26, 13.33s/it]

(0.7243383249172907, 0.5029818771350819, 0.6803728070175439, 0.5783814374646293)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 55 epochs.


  7%|▋         | 2/30 [00:16<03:18,  7.09s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [02:02<00:04,  4.17s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [02:06<00:00,  4.23s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [27]:
X_train = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_train]
X_val = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_val]
X_test = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_test]

In [28]:
from sklearn.model_selection import ParameterGrid

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_tfidf = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_tfidf.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_tfidf.json', 'w') as f:
    json.dump(scores_tfidf, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_591361/798717303.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 77 epochs.


  3%|▎         | 1/30 [00:28<13:45, 28.46s/it]

(0.7108654013581752, 0.4834869431643625, 0.5916353383458647, 0.5321217244294167)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 60 epochs.


  7%|▋         | 2/30 [00:33<06:43, 14.40s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_591361/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
 97%|█████████▋| 29/30 [02:58<00:05,  5.91s/it]

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [03:04<00:00,  6.15s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [29]:
X_train = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_train]
X_val = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_val]
X_test = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_w2v = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_w2v.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_w2v.json', 'w') as f:
    json.dump(scores_w2v, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_591361/3047588890.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 78 epochs.


  3%|▎         | 1/30 [00:13<06:17, 13.00s/it]

(0.5902620581577572, 0.390028686589927, 0.8412437343358395, 0.5329595355645421)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_591361/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
  7%|▋         | 2/30 [00:15<03:11,  6.83s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [01:41<00:03,  3.49s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:45<00:00,  3.52s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


## Text + HandCrafted Features

In [30]:
def get_features(d, BoW=False, tfidf=False, w2v=False):
    a = get_text_features(d, BoW, tfidf, w2v)
    a.extend(get_meta_features(d, True, True, True, True))
    return a

In [31]:
X_train = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_train]
X_val = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_val]
X_test = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_bow_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_bow_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_bow_hand.json', 'w') as f:
    json.dump(scores_bow_hand, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_591361/49499707.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 55 epochs.


  3%|▎         | 1/30 [00:14<07:13, 14.94s/it]

(0.6376675953334494, 0.401663033007149, 0.6204573934837093, 0.4876427318334308)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 100 epochs.


  7%|▋         | 2/30 [00:17<03:41,  7.90s/it]

...

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [01:54<00:04,  4.02s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:58<00:00,  3.95s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [32]:
X_train = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_train]
X_val = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_val]
X_test = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_tfidf_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_tfidf_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_tfidf_hand.json', 'w') as f:
    json.dump(scores_tfidf_hand, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_591361/940750581.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 16 epochs.


  3%|▎         | 1/30 [00:27<13:30, 27.96s/it]

(0.49259968657496084, 0.3149136357253195, 0.7025375939849624, 0.4348880054300398)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 32 epochs.


  7%|▋         | 2/30 [00:32<06:30, 13.94s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_591361/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
 97%|█████████▋| 29/30 [02:56<00:05,  6.00s/it]

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [03:02<00:00,  6.08s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [33]:
X_train = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_train]
X_val = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_val]
X_test = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_w2v_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_w2v_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_w2v_hand.json', 'w') as f:
    json.dump(scores_w2v_hand, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_591361/1087097735.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 32 epochs.


  3%|▎         | 1/30 [00:14<06:53, 14.24s/it]

(0.6775639909454989, 0.29782608695652174, 0.11802944862155389, 0.16905990576621047)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 25 epochs.


  7%|▋         | 2/30 [00:16<03:20,  7.15s/it]

...

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [01:43<00:03,  3.60s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:47<00:00,  3.59s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
